# Data Collection

In [ ]:
import pandas as pd
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import time

service = Service("chrome_driver/chromedriver.exe")
driver = webdriver.Chrome(service=service)

driver.get("https://vsmobile.bet9ja.com/mobile/themes/?sk=bet9ja&t=b61c29e6-9348-4c58-af90-378760a74693&game"
               "=league_premier&pid=14001,14003,14011,14012,14014,14015,14016,"
               "14017&v=0&text=Premier&lang=en_GB#resutls&ui_state=dialog")
# driver.maximize_window()
time.sleep(10)

action = ActionChains(driver)
record_list = []

print("Games Loading  ...\n")
while True:
    week_num = driver.find_element(By.XPATH, "//span[@id='leagueWeekNumber']").text
    counter = driver.find_element(By.XPATH, "//div[@id='bets-time-betContdown']").text

    if counter[:2] == "00" and int(counter[-2:]) < 50 and int(counter[-2:]) > 20:
        match_result = driver.find_element(By.XPATH, "//div[@id='tab_id_Match_Result']").text

        driver.find_element(By.XPATH, "//a[@id='ui-id-3']").click()
        over_one = driver.find_element(By.XPATH, "//div[@id='tab_id_Over_Under_1_5']").text

        market_select = driver.find_element(By.XPATH, "//select[@id='bet-select-market']")

        market_select.click()
        arrow_down = [ActionChains(driver).send_keys(Keys.ARROW_DOWN).perform() for _ in range(3)]
        action.send_keys(Keys.ENTER).perform()
        over_two = driver.find_element(By.XPATH, "//div[@id='tab_id_Over_Under_2_5']").text

        market_select.click()
        arrow_down = [ActionChains(driver).send_keys(Keys.ARROW_DOWN).perform() for _ in range(4)]
        action.send_keys(Keys.ENTER).perform()
        over_three = driver.find_element(By.XPATH, "//div[@id='tab_id_Over_Under_3_5']").text

        market_select.click()
        arrow_down = [ActionChains(driver).send_keys(Keys.ARROW_DOWN).perform() for _ in range(5)]
        action.send_keys(Keys.ENTER).perform()
        over_four = driver.find_element(By.XPATH, "//div[@id='tab_id_Over_Under_4_5']").text
        # print("counting")
        time.sleep(70)

        while True:
            counter = driver.find_element(By.XPATH, "//div[@id='bets-time-betContdown']").text
            current_week = driver.find_element(By.XPATH, "//span[@id='leagueWeekNumber']").text
        
            if counter[:2] == "00" and int(counter[-2:]) < 58 and int(counter[-2:]) > 50:
                week_check = int(current_week) - 1
                if week_check == 0:
                    current_week = 39
                # print("collecting ...")
                driver.find_element(By.XPATH, "//div[@class='ui-panel-wrapper']//i[@class='fa fa-bars icon-menu']").click()
                time.sleep(1)
                driver.find_element(By.XPATH, "//a[@id='a_bet_results']").click()
                time.sleep(8)
                try:                  
                    score_result = driver.find_element(By.XPATH, "//table[@id='results-div-header-mainTable']").text
                    driver.find_element(By.XPATH, "//div[@id='results']//i[@class='fa fa-bars icon-menu']").click()
                except ElementClickInterceptedException:
                    print("Network Delay")
                    time.sleep(20)
                    score_result = driver.find_element(By.XPATH, "//table[@id='results-div-header-mainTable']").text
                    driver.find_element(By.XPATH, "//div[@id='results']//i[@class='fa fa-bars icon-menu']").click()
                
                time.sleep(1)
                driver.find_element(By.XPATH, "//li[@class='li_bet']//a[@id='a_bet_bet']").click()
                time.sleep(3)
                break
            else:
                time.sleep(1)   

        match_result = match_result.split("\n")[2::2]
        over_one = over_one.split("\n")[2::2]
        over_two = over_two.split("\n")[2::2]
        over_three = over_three.split("\n")[2::2]
        over_four = over_four.split("\n")[2::2]
        score_result = score_result.split(f"WEEK {int(current_week) - 1}")[1].split("\n")[1:11]

        for hda, one, two, three, four, score in zip(match_result, over_one, over_two, over_three, over_four, score_result):
            data = (int(current_week) - 1,
                    hda[:3], hda[6:9],
                    float(hda[-14:-10]), float(hda[-9:-5]), float(hda[-4:]),
                    float(one[-9:-5]), float(one[-4:]),
                    float(two[-9:-5]), float(two[-4:]),
                    float(three[-9:-5]), float(three[-4:]),
                    float(four[-9:-5]), float(four[-4:]),
                    int(score[4]), int(score[6]))
            record_list.append(data)
            
        print("Week", int(current_week) - 1, "Data Collection Complete ...")

        if int(current_week) == 39:
            new_df = pd.DataFrame(record_list, columns=[
                "week",
                "HT", "AT", "home", "draw", "away", 
                "over_1", "under_1", 
                "over_2", "under_2",
                "over_3", "under_3",
                "over_4", "under_4",
                "H_score", "A_score"])
            new_df.to_csv("odds_data.csv", index=False)
    else:
        time.sleep(1)


# # Close the browser session
# driver.quit()

In [ ]:
new_df = pd.DataFrame(record_list, columns=[
    "week",
    "HT", "HT", "home", "draw", "away", 
    "over_1", "under_1", 
    "over_2", "under_2",
    "over_3", "under_3",
    "over_4", "under_4",
    "H_score", "A_score"])

new_df.to_csv("test_12_odds_data.csv", index=False)

# Analyzing the data

In [41]:
import pandas as pd

df = pd.read_csv("saved_data/test_13_odds_data.csv") # odds_data
df["T_goals"] = df["H_score"] + df["A_score"]
df

,week,HT,HT.1,home,draw,away,over_1,under_1,over_2,under_2,over_3,under_3,over_4,under_4,H_score,A_score,T_goals
0,36,FOR,BUR,1.95,3.29,4.25,1.33,3.33,2.08,1.75,3.83,1.26,9.27,1.06,1,1,2
1,36,NWC,EVE,1.65,3.63,5.82,1.21,4.47,1.73,2.11,2.87,1.42,6.30,1.12,2,0,2
2,36,MNU,ARS,2.86,3.54,2.38,1.24,4.04,1.80,2.02,3.13,1.36,6.99,1.10,1,0,1
3,36,TOT,BRI,2.38,3.20,3.13,1.26,3.84,1.88,1.92,3.26,1.34,7.47,1.09,3,3,6
4,36,MNC,CRY,1.42,4.55,7.93,1.19,4.74,1.63,2.27,2.69,1.47,5.54,1.14,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2315,3,FOR,EVE,2.29,3.02,3.53,1.33,3.35,2.07,1.75,3.76,1.27,9.27,1.06,0,0,0
2316,3,BRN,FUL,1.77,3.43,5.15,1.27,3.78,1.89,1.91,3.29,1.33,7.54,1.08,3,0,3
2317,3,MNU,LIV,2.43,3.24,3.02,1.24,4.01,1.83,1.98,3.14,1.36,7.10,1.09,1,2,3
2318,3,TOT,CHE,2.48,3.20,2.98,1.26,3.83,1.88,1.92,3.26,1.34,7.53,1.09,1,0,1


In [46]:
check_list = []
odd_list = []
for val_1, val_2 in zip(df["over_3"], df["T_goals"]):
    check_list.append((val_1, val_2))
    if len(check_list) == 10:
        # print(check_list)
        sorted_odds = sorted(check_list, key=lambda x: x[0], reverse=True)
        all_odds = [x[0] for x in sorted_odds]
        for odd in sorted_odds:
            if all_odds.count(odd[0]) == 1:
                odd_list.append(odd)
                break               
        check_list = []

print("Total Stake: ", len(odd_list))
# odd_list

Total Stake:  232


In [48]:
count = 1
martingale_list = [4, 5, 2, 2, 3, 1, 1, 2, 6, 1, 1, 1, 1, 9, 8]
# for x in odd_list:
#     if x[1] > 3:
#         # print(count)
#         martingale_list.append(count)
#         count = 1       
#     else:
#         count += 1
        
print(martingale_list)
print(sorted(martingale_list, reverse=True)[:50])
print("Hightest Martingale Level:", max(martingale_list))

[4, 5, 2, 2, 3, 1, 1, 2, 6, 1, 1, 1, 1, 9, 8]
[9, 8, 6, 5, 4, 3, 2, 2, 2, 1, 1, 1, 1, 1, 1]
Hightest Martingale Level: 9


In [49]:
win_length = sorted(martingale_list, reverse=True)
total_diff = 0
for x in win_length:
    if x > 5:
        total_diff += 1
    else:
        break
print("Losses:", total_diff)
print("Wins:", len(martingale_list) - total_diff)

Losses: 3
Wins: 12
